In [1]:
import pandas as pd
import numpy as np
import glob as glob
import os
import re
from rdkit.Chem import rdchem
from multiprocessing import Pool

import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
# G = nx.read_graphml('eom-net_0p1_new-rem-blink_k3.graphml')
# G = nx.read_graphml('eom-net_0p01_lbl.graphml')



from rdkit.Chem import rdFMCS
from rdkit.Chem import MolFromSmiles, MolFromSmarts, MolToInchiKey
from rdkit.Chem.Draw import MolsToGridImage

import shutil
import json
from collections import Counter
import numpy as np
import pandas as pd
from rdkit import Chem
import os

sys.path.insert(0,'/global/homes/b/bpb/repos/blink')
import blink
sys.path.insert(0,'/global/homes/b/bpb/repos/scndb/')
import scndb.tools as wt

sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')
from metatlas.io import feature_tools as ft

In [2]:

smiles_col = 'smiles_identity'

# Maximum allowed distance between identity hits for clustering
max_node_distance = 2

In [3]:
df = pd.read_pickle('top_scoring_inchikey.pkl')
all_df = pd.read_pickle('/global/cfs/cdirs/metatlas/projects/spectral_libraries/temp_backup.pkl')

# df = pd.read_pickle('top_scoring_inchikey_lbl.pkl')
# all_df = pd.read_pickle('/global/cfs/cdirs/metatlas/projects/spectral_libraries/temp_backup_lbl.pkl')

all_df['nl_spectrum'] = all_df.apply(lambda x: np.asarray([x['mdm_mz_vals'],x['mdm_i_vals']]),axis=1)

all_df['sum_frag_intensity'] = all_df['mdm_i_vals'].apply(lambda x: np.sum(x))
all_df['original_spectrum'] = all_df['nl_spectrum']#df.apply(lambda x: np.asarray([x['original_mz_vals'],x['original_i_vals']]),axis=1)
all_df = all_df[all_df['sum_frag_intensity']>0]
all_df = all_df[pd.notna(all_df['predicted_formula'])]
all_df = all_df[all_df['massive_id'].isin(df['massive_id'])]
df['num_mdm_frags'] = df['mdm_mz_vals'].apply(lambda x: len(x))
all_df

,no_extension,h5,buddy,data_dir,massive_id,h5_basename,no_extension_basename,redu_filename,title,dataset,...,original_i_vals,predicted_formula,estimated_fdr,predicted_mass,mass_error,ms2_mz_vals,ms2_i_vals,nl_spectrum,sum_frag_intensity,original_spectrum
0,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive,MSV000088543,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,f.MSV000088543/ccms_peak/raw_data/20181019_JJ_...,Exometabolomics of Switchgrass rhizosphere,MSV000088543,...,"[462166.65625, 206324.203125, 276785.5, 56654....",C7H14N2O4S,0.000000,221.060152,-0.000458,NaN,NaN,"[[177.0698699951172, 193.06468200683594, 195.0...",1.238616e+07,"[[177.0698699951172, 193.06468200683594, 195.0..."
1,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive,MSV000088543,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,f.MSV000088543/ccms_peak/raw_data/20181019_JJ_...,Exometabolomics of Switchgrass rhizosphere,MSV000088543,...,"[38185.5859375, 5534106.5, 2194544.25, 80258.8...",CH7N4O3P,0.000000,153.018301,-0.000512,NaN,NaN,"[[65.03784942626953, 91.01720428466797, 109.02...",8.319949e+06,"[[65.03784942626953, 91.01720428466797, 109.02..."
2,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive,MSV000088543,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,f.MSV000088543/ccms_peak/raw_data/20181019_JJ_...,Exometabolomics of Switchgrass rhizosphere,MSV000088543,...,"[727181.625, 44015.3125, 42669.6484375, 49133....",C8H4OS2,0.000000,178.963081,0.000839,NaN,NaN,"[[55.017250061035156, 71.0121841430664, 176.94...",8.202274e+06,"[[55.017250061035156, 71.0121841430664, 176.94..."
3,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive,MSV000088543,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,f.MSV000088543/ccms_peak/raw_data/20181019_JJ_...,Exometabolomics of Switchgrass rhizosphere,MSV000088543,...,"[6075.94775390625, 3627.46142578125, 12005.785...",C10H10S,0.000000,161.043045,0.000926,NaN,NaN,"[[85.02783966064453, 99.04350280761719, 119.03...",3.716612e+06,"[[85.02783966064453, 99.04350280761719, 119.03..."
4,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000088543/cc...,/pscratch/sd/b/bpb/massive,MSV000088543,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,20181019_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...,f.MSV000088543/ccms_peak/raw_data/20181019_JJ_...,Exometabolomics of Switchgrass rhizosphere,MSV000088543,...,"[156714.0625, 887873.125, 805822.5, 3488263.75...",H3N7O2,0.000000,132.027546,0.000994,NaN,NaN,"[[59.011905670166016, 88.03870391845703, 114.0...",3.663153e+06,"[[59.011905670166016, 88.03870391845703, 114.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074769,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,CenturyExpPilot_20230414,20230919_EB_SMK_107002-011_CenturyExpPilot_202...,20230919_EB_SMK_107002-011_CenturyExpPilot_202...,NaN,NaN,NaN,...,"[35006.5546875, 12811.0244140625, 15123.492187...",C20H16O9,0.001499,399.072156,-0.000177,NaN,NaN,"[[207.08123779296875, 235.0756072998047, 267.1...",2.703867e+04,"[[207.08123779296875, 235.0756072998047, 267.1..."
1074770

In [4]:
# FILTER OUT ANY SPECTRA THAT DO NOT HAVE MS1 OR NOT CALIBRATED WELL

cols = ['precursor_mz',  'rt']
g = [(_,gg[cols]) for _,gg in all_df.groupby('h5')]


def make_atlas(df,ppm_tolerance = 5,mz_tol = 0.002,extra_rt=1):
    atlas = df.copy()
    # atlas.reset_index(inplace=True,drop=True)
    atlas.index.name = 'label'
    atlas.reset_index(inplace=True,drop=False)
    atlas.rename(columns={'rt':'rt_peak','precursor_mz':'mz'},inplace=True)
    atlas['rt_min'] = atlas['rt_peak'] - extra_rt
    atlas['rt_max'] = atlas['rt_peak'] + extra_rt
    atlas['mz_tolerance'] = mz_tol
    # atlas['rt_tolerance'] = 100
    atlas['ppm_tolerance'] = ppm_tolerance
    atlas['extra_time'] = 0
    atlas['group_index'] = ft.group_consecutive(atlas['mz'].values[:],
                                        stepsize=ppm_tolerance,
                                        do_ppm=True)
    return atlas




def process_row(gg):
    filename = gg[0]
    atlas = make_atlas(gg[1])
    try:
        d = ft.get_atlas_data_from_file(filename,atlas,desired_key='ms1_neg')
    except:
        print('Can not read',filename)
    d = d[d['in_feature']==True].groupby('label').apply(ft.calculate_ms1_summary).reset_index()
    d['filename'] = filename
    return d
    
with Pool(20) as pool:
    out = pool.map(process_row,g)    
out = pd.concat(out)




temp = pd.merge(out,all_df['precursor_mz'],left_on='label',right_index=True,how='inner')
temp['ppm_error'] = abs(temp['mz_centroid']-temp['precursor_mz']) / temp['precursor_mz'] * 1e6
temp = temp[temp['ppm_error']<5]
temp = temp[temp['num_datapoints']>30]
# fig,ax = plt.subplots()
# temp['num_datapoints'].apply(np.log10).hist(bins=100,ax=ax)


all_df = all_df[all_df.index.isin(temp['label'])]

In [5]:
print(all_df.shape)
# all_df = all_df[~all_df['predicted_formula'].isin(df['predicted_formula'])]
# print(all_df.shape)

(604904, 50)


In [6]:
mz_tol = 0.002
similarity_cutoff = 0.2
isolation_tol = 0.5
min_intensity_ratio = 2 #minimum intensity ratio between sum of fragments between two spectra
my_polarity = 'negative' #either 'positive' or 'negative'

deltas = pd.read_csv('../data/mdm_neutral_losses.csv')

all_df['round3_precursor_mz'] = all_df['precursor_mz'].round(3)
g = all_df.groupby('round3_precursor_mz')
# g = all_df.groupby('predicted_formula')
gg = [g.get_group(x) for x in g.groups]

out = []
for ggg in gg:
    if ggg.shape[0]>1:
        temp = wt.eliminate_duplicate_spectra(ggg,deltas,mz_tol=mz_tol,
                                similarity_cutoff=similarity_cutoff,
                                min_intensity_ratio=min_intensity_ratio)
        out.append(temp)
        # print(temp.shape[0],ggg.shape[0])
    else:
        out.append(ggg)
        
out = pd.concat(out)
out.reset_index(inplace=True,drop=True)
out


,no_extension,h5,buddy,data_dir,massive_id,h5_basename,no_extension_basename,redu_filename,title,dataset,...,predicted_formula,estimated_fdr,predicted_mass,mass_error,ms2_mz_vals,ms2_i_vals,nl_spectrum,sum_frag_intensity,original_spectrum,round3_precursor_mz
0,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,SoilWetDry_final,20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_...,20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_...,NaN,NaN,NaN,...,C4H6O2,0.000000e+00,85.029503,-0.000238,NaN,NaN,"[[55.018558502197266, 57.034358978271484, 71.0...",6.713097e+04,"[[55.018558502197266, 57.034358978271484, 71.0...",85.029
1,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,SoilWetDry_final,20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_...,20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_...,NaN,NaN,NaN,...,C3H4O3,0.000000e+00,87.008768,-0.000168,NaN,NaN,"[[58.00577926635742, 59.013587951660156, 72.99...",5.343726e+04,"[[58.00577926635742, 59.013587951660156, 72.99...",87.009
2,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,FRCgw2_20230830,20230831_EB_MdR_101544-059_FRCgw2_20230830_EXP...,20230831_EB_MdR_101544-059_FRCgw2_20230830_EXP...,NaN,NaN,NaN,...,C2H2O4,0.000000e+00,88.988033,0.000039,NaN,NaN,"[[44.99815368652344, 60.993160247802734, 87.00...",5.457970e+05,"[[44.99815368652344, 60.993160247802734, 87.00...",88.988
3,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,FRCgw2_20230830,20230831_EB_MdR_101544-059_FRCgw2_20230830_EXP...,20230831_EB_MdR_101544-059_FRCgw2_20230830_EXP...,NaN,NaN,NaN,...,C3H6O3,0.000000e+00,89.024418,0.000028,NaN,NaN,"[[44.99815368652344, 59.01304626464844, 59.013...",5.615366e+05,"[[44.99815368652344, 59.01304626464844, 59.013...",89.024
4,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,/global/cfs/cdirs/metatlas/projects/rawdata_fo...,SoilWetDry_final,20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_...,20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_...,NaN,NaN,NaN,...,C4H4O3,0.000000e+00,99.008768,-0.000274,NaN,NaN,"[[55.018524169921875, 68.9980239868164, 71.013...",7.759353e+04,"[[55.018524169921875, 68.9980239868164, 71.013...",99.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18484,/pscratch/sd/b/bpb/massive/v01/MSV000091211/pe...,/pscratch/sd/b/bpb/massive/v01/MSV000091211/pe...,/pscratch/sd/b/bpb/massive/v01/MSV000091211/pe...,/pscratch/sd/b/bpb/massive,MSV000091211,NEG_21.h5,NEG_21,f.MSV000091211/peak/MZml/NEG_21,20230206 - Lignin ligands-quinones-permafrost...,MSV000091211,...,C46H71N3O25,3.298795e-09,1064.430389,0.000112,NaN,NaN,"[[1002.4296875, 1028.40966796875, 1046.4196777...",3.710669e+06,"[[1002.4296875, 1028.40966796875, 1046.4196777...",1064.431
18485,/pscratch/sd/b/bpb/massive/v03/MSV000086931/pe...,/pscratch/sd/b/bpb/massive/v03/MSV000086931/pe...,/pscratch/sd/b/bpb/massive/v03/MSV000086931/pe...,/pscratch/sd/b/bpb/massive,MSV000086931,SFA_90m_R3_soil_lipids_Neg_20Feb20_Lola-WCSH61...,SFA_90m_R3_soil_lipids_Neg_20Feb20_Lola-WCSH61...,f.MSV000086931/peak/GNPS mzml files_lipids arc...,GNPS- Influence of drought and wet-up on the s...,MSV000086931,...,C61H99N11O7,0.000000e+00,1096.765619,0.000019,NaN,NaN,"[[1024.7841796875, 1052.7757568359375, 1053.76...",1.020815e+05,"[[1024.7841796875, 1052.7757568359375, 1053.76...",1096.766
18486,/pscratch/sd/b/bpb/massive/v03/M

In [7]:
out = wt.eliminate_duplicate_spectra(out,deltas,mz_tol=mz_tol,
                                similarity_cutoff=similarity_cutoff,
                                min_intensity_ratio=min_intensity_ratio)
out.reset_index(inplace=True,drop=True)
out.shape[0]

9198

In [8]:
df.reset_index(inplace=True,drop=True)
out.reset_index(inplace=True,drop=True)
temp = pd.concat([df,out],axis=0)
temp.reset_index(inplace=True,drop=True)
mass_diffs = [0] + deltas['mass'].tolist()
rem_df = wt.do_remblink_networking(temp,temp,mass_diffs=mass_diffs,
                                   spectra_attr='nl_spectrum')
# rem_df = wt.do_remblink_networking(out,df,spectra_attr='nl_spectrum')
rem_df.rename(columns={'rem_predicted_score':'rem_blink_score'},inplace=True)
cols = ['ref','query','rem_blink_score']
rem_df = rem_df[cols]
print(rem_df.shape)
rem_df = rem_df[rem_df['rem_blink_score']>0.1]
print(rem_df.shape)

Using /global/homes/b/bpb/repos/blink/models/mdm_negative_random_forest.joblib
(16093016, 3)
(88271, 3)


In [9]:
p = wt.get_formula_props(temp,formula_key='predicted_formula')
temp = pd.merge(temp,p,left_on='predicted_formula',right_on='formula',how='left')

In [10]:
if 'original_index' in temp.columns:
    temp.drop(columns=['original_index'],inplace=True)
temp.index.name = 'original_index'
temp.reset_index(inplace=True,drop=False)


In [11]:
p.columns

Index(['dbe', 'dbe_ai', 'dbe_ai_mod', 'ai_mod', 'ai', 'nosc', 'h_to_c',
       'o_to_c', 'n_to_c', 'p_to_c', 'n_to_p', 'c', 'h', 'o', 'n', 's', 'p',
       'formula'],
      dtype='object')

In [12]:
cols = ['original_index', 
       'massive_id', 'no_extension_basename','title', 'description', 
       'precursor_mz', 'isolated_precursor_mz', 'rt',
       'coisolated_precursor_count', 'predicted_formula',
       'estimated_fdr', 'predicted_mass', 'mass_error', 'num_mdm_frags',
       'max_score_identity', 'best_match_method_identity',
       'max_matches_identity', 'name_identity', 'inchi_key_identity',
       'smiles_identity', 'formula_identity', 'precursor_mz_identity',
       'original_p2d2_index_identity', 'isin_fticr_formula']
cols = cols + ['dbe', 'dbe_ai', 'dbe_ai_mod', 'ai_mod', 'ai', 'nosc', 'h_to_c',
       'o_to_c', 'n_to_c', 'p_to_c', 'n_to_p', 'c', 'h', 'o', 'n', 's', 'p']
# 'instrument', 'keywords', 'ChromatographyAndPhase',
#        'YearOfAnalysis', 'SampleType', 'IonizationSourceAndPolarity',
#        'SampleCollectionMethod', 'MassSpectrometer',
#        'NCBITaxonomy', 'SampleExtractionMethod', 

In [13]:
import networkx as nx
# # Create the graph from the similarity matrix
G = nx.from_pandas_edgelist(rem_df,source='ref',target='query',edge_attr='rem_blink_score')
# (similarity_matrix > similarity_cutoff)
# # Add node data from ms2_df to G
# cols = list(set([c for c in temp.columns if not 'spect' in c]) - set(['obs','coisolated_precursor_mz_list','mol','filename', 'basename', 'experiment']))
# cols = [c for c in cols if not 'mdm_' in c]
# cols = [c for c in cols if not 'original_' in c]
# cols = [c for c in cols if not 'nl_' in c]

# cols = cols + ['stable']
node_data = temp[cols].fillna('').to_dict(orient='index')






nx.set_node_attributes(G, node_data)

# print(len(G.nodes))
# print(len(G.edges))

# # Remove self-loops
G.remove_edges_from([(u, v) for u, v in G.edges() if u == v])         

# Remove isolates
G.remove_nodes_from(list(nx.isolates(G)))
print('nodes',len(G.nodes))
print('edges',len(G.edges))


nodes 7359
edges 48261


In [14]:

# Get identity hits smiles for all nodes
identity_smiles = nx.get_node_attributes(G, smiles_col)
node_id = [k for k, v in identity_smiles.items() if isinstance(v,str)]
identity_smiles = [v for k, v in identity_smiles.items() if isinstance(v,str)]
df = pd.DataFrame(data=identity_smiles,columns=['smiles'])
df['node_id'] = node_id
df['mol'] = df['smiles'].apply(lambda x: MolFromSmiles(x) if x is not '' else None)
df['inchi_key'] = df['mol'].apply(lambda x: MolToInchiKey(x) if x is not None else None)

df

,smiles,node_id,mol,inchi_key
0,CC(C)CC(N=C(O)C(O)C(C)C)C(=O)O,1033.0,<rdkit.Chem.rdchem.Mol object at 0x7f6d57e85350>,SWEPQJJUAVEABL-UHFFFAOYSA-N
1,CCCCCCCC(O)=NCC(=O)O,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f6d57e85200>,SAVLIIGUQOSOEP-UHFFFAOYSA-N
2,,3055.0,None,None
3,,4033.0,None,None
4,,5672.0,None,None
...,...,...,...,...
7354,,10391.0,None,None
7355,,10455.0,None,None
7356,,8425.0,None,None
7357,,10612.0,None,None


In [15]:
import requests
import json

url = "https://npclassifier.gnps2.org/classify"
df.reset_index(inplace=True,drop=True)
out = []
for i,row in df[pd.notna(df['inchi_key'])].iterrows(): # for some reason smiles is "" and not None
    # smiles = df.loc[0,'smiles_identity']
    r = requests.get(url, params={"smiles": row['smiles']})
    if r.status_code == 200:
        d = r.json()
    else:
        if 'C' in row['smiles']:
            print(row['smiles'])
        d = {'class_results': None,
                     'superclass_results': None,
                     'pathway_results': None,
                     'isglycoside': False}
    d['inchi_key'] = row['inchi_key']
    out.append(d)

In [16]:
out2 = pd.DataFrame(out)
cols = ['class_results', 'superclass_results', 'pathway_results']
for c in cols:
    out2[c] = out2[c].apply(lambda x: ','.join(sorted(x)) if isinstance(x,list) else '')
out2

,class_results,superclass_results,pathway_results,isglycoside,inchi_key
0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids,False,SWEPQJJUAVEABL-UHFFFAOYSA-N
1,Oxo fatty acids,Fatty Acids and Conjugates,Fatty acids,False,SAVLIIGUQOSOEP-UHFFFAOYSA-N
2,,,Shikimates and Phenylpropanoids,False,KKADPXVIOXHVKN-UHFFFAOYSA-N
3,Cinnamic acids and derivatives,Phenylpropanoids (C6-C3),Shikimates and Phenylpropanoids,False,BTNMPGBKDVTSJY-UHFFFAOYSA-N
4,Simple phenolic acids,Phenolic acids (C6-C1),Shikimates and Phenylpropanoids,False,PNYWALDMLUDDTA-UHFFFAOYSA-N
...,...,...,...,...,...
1378,"Shikimic acids and derivatives,Simple phenolic...",Phenolic acids (C6-C1),Shikimates and Phenylpropanoids,False,QFOHBWFCKVYLES-UHFFFAOYSA-N
1379,Valerenane sesquiterpenoids,Sesquiterpenoids,Terpenoids,False,VBBXZFLAYWAXSK-HYJCDKNOSA-N
1380,Colensane and Clerodane diterpenoids,Diterpenoids,Terpenoids,False,OLDFOHABNQZMKB-UHFFFAOYSA-N
1381,Oleanane triterpenoids,Triterpenoids,Terpenoids,False,QGWDYPREORDRIT-UHFFFAOYSA-N


In [17]:
def replace_list_identifiers(df,identifier='class_results'):
    temp = df.loc[pd.notna(df[identifier]),identifier].tolist()
    temp = [x.split(',') for x in temp]
    temp = [item for sublist in temp for item in sublist]
    temp = pd.Series(temp)
    pathway_counts = temp.value_counts()
    for i,row in df[pd.notna(df[identifier])].iterrows():
        split_results = row[identifier].split(',')
        if len(split_results) > 1:
            d = {}
            for r in split_results:
                d[r] = pathway_counts[r]
            df.at[i, identifier] = max(d, key=d.get)

    return df

out2 = replace_list_identifiers(out2,identifier='class_results')
out2 = replace_list_identifiers(out2,identifier='pathway_results')
out2 = replace_list_identifiers(out2,identifier='superclass_results')
out2

,class_results,superclass_results,pathway_results,isglycoside,inchi_key
0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids,False,SWEPQJJUAVEABL-UHFFFAOYSA-N
1,Oxo fatty acids,Fatty Acids and Conjugates,Fatty acids,False,SAVLIIGUQOSOEP-UHFFFAOYSA-N
2,,,Shikimates and Phenylpropanoids,False,KKADPXVIOXHVKN-UHFFFAOYSA-N
3,Cinnamic acids and derivatives,Phenylpropanoids (C6-C3),Shikimates and Phenylpropanoids,False,BTNMPGBKDVTSJY-UHFFFAOYSA-N
4,Simple phenolic acids,Phenolic acids (C6-C1),Shikimates and Phenylpropanoids,False,PNYWALDMLUDDTA-UHFFFAOYSA-N
...,...,...,...,...,...
1378,Simple phenolic acids,Phenolic acids (C6-C1),Shikimates and Phenylpropanoids,False,QFOHBWFCKVYLES-UHFFFAOYSA-N
1379,Valerenane sesquiterpenoids,Sesquiterpenoids,Terpenoids,False,VBBXZFLAYWAXSK-HYJCDKNOSA-N
1380,Colensane and Clerodane diterpenoids,Diterpenoids,Terpenoids,False,OLDFOHABNQZMKB-UHFFFAOYSA-N
1381,Oleanane triterpenoids,Triterpenoids,Terpenoids,False,QGWDYPREORDRIT-UHFFFAOYSA-N


In [18]:
df = pd.merge(df,out2,on='inchi_key',how='left')
df.reset_index(inplace=True,drop=True)

In [19]:

df

,smiles,node_id,mol,inchi_key,class_results,superclass_results,pathway_results,isglycoside
0,CC(C)CC(N=C(O)C(O)C(C)C)C(=O)O,1033.0,<rdkit.Chem.rdchem.Mol object at 0x7f6d57e85350>,SWEPQJJUAVEABL-UHFFFAOYSA-N,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids,False
1,CCCCCCCC(O)=NCC(=O)O,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f6d57e85200>,SAVLIIGUQOSOEP-UHFFFAOYSA-N,Oxo fatty acids,Fatty Acids and Conjugates,Fatty acids,False
2,,3055.0,None,None,NaN,NaN,NaN,NaN
3,,4033.0,None,None,NaN,NaN,NaN,NaN
4,,5672.0,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7354,,10391.0,None,None,NaN,NaN,NaN,NaN
7355,,10455.0,None,None,NaN,NaN,NaN,NaN
7356,,8425.0,None,None,NaN,NaN,NaN,NaN
7357,,10612.0,None,None,NaN,NaN,NaN,NaN


In [20]:
for c in ['class_results', 'superclass_results', 'pathway_results']:
    nx.set_node_attributes(G, df.set_index('node_id')[c].fillna('').to_dict(), c)


In [21]:
nx.write_graphml(G,'CarbonNetwork_noprop.graphml')
G_filt = nx.maximum_spanning_tree(G)
nx.write_graphml(G_filt, 'CarbonNetwork_mst_noprop.graphml')

In [22]:
G = nx.read_graphml('CarbonNetwork_noprop.graphml')
# main_column = 'pathway_results'
main_column = 'class_results'
all_columns = ['class_results', 'superclass_results', 'pathway_results']
df = G.nodes(data=True) 
df = pd.DataFrame(df, columns=['node_id', 'data'])
df.set_index('node_id', inplace=True)
df = df['data'].apply(pd.Series)
df.reset_index(inplace=True,drop=False)
df.head()

,node_id,original_index,massive_id,no_extension_basename,precursor_mz,isolated_precursor_mz,rt,coisolated_precursor_count,predicted_formula,estimated_fdr,...,o,n,s,p,class_results,superclass_results,pathway_results,title,description,n_to_p
0,1033.0,1033,FRCgwater_20230105,20230113_EB_MdR_101544-059_FRCgwater_20230105_...,230.139208,230.139282,2.633,1.0,C11H21NO4,0.0,...,4.0,1.0,0.0,0.0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids,NaN,NaN,NaN
1,0.0,0,MSV000092487,S56_neg_2,200.128534,200.128876,6.159,1.0,C10H19NO3,0.0,...,3.0,1.0,0.0,0.0,Oxo fatty acids,Fatty Acids and Conjugates,Fatty acids,GNPS - Neckar and Spree River Water DOM,Non-target metabolomics of River DOM samples i...,NaN
2,3055.0,3055,FRCgw2_20230830,20230831_EB_MdR_101544-059_FRCgw2_20230830_EXP...,200.129083,200.128906,3.595,2.0,C10H19NO3,0.0,...,3.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4033.0,4033,FRCgw2_20230830,20230831_EB_MdR_101544-059_FRCgw2_20230830_EXP...,230.139598,230.139572,2.626,1.0,C11H21NO4,0.0,...,4.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5672.0,5672,MSV000088823,DOM_Interlab-LCMS_Lab024_M_NEG_MS2_rep2,230.148272,230.148209,16.316,1.0,C6H17N9O,0.0,...,1.0,9.0,0.0,0.0,NaN,NaN,NaN,GNPS DOM LC-MS/MS Interlab Comparison 2020 COM...,Interlab Study of LC-MS/MS analyis of Marine D...,NaN


In [23]:

# def get_most_common(classes):
#     class_counts = Counter(classes)
#     # Get the most common string
#     most_common_class = class_counts.most_common(1)[0][0]
#     return most_common_class

# classyfire_dir = '/global/cfs/cdirs/metatlas/projects/classyfire_annotations/'
# superclass_name = []
# class_name = []
# subclass_name = []
# iks = df.loc[pd.notna(df['inchi_key']),'inchi_key'].unique()
# for ik in iks:
#     f = '%s.json'%ik
#     f = os.path.join(classyfire_dir,f)
#     with open(f,'r') as fid:
#         cf = fid.read()
#     cf = json.loads(cf.strip())
#     if isinstance(cf,str):
#         cf = json.loads(cf)
#     # cf_json = json.dumps(cf)
#     if not main_column in cf:
#         cf[main_column] = {'name':None}
#     if not 'superclass' in cf:
#         cf['superclass'] = {'name':None}
#         cf['class'] = {'name':None}
#     if cf[main_column] is None:
#         cf[main_column] = {'name':None}
#     superclass_name.append(cf['superclass']['name'])
#     class_name.append(cf['class']['name'])
#     subclass_name.append(cf[main_column]['name'])

# temp = pd.DataFrame(index=iks)
# temp['superclass'] = superclass_name
# temp['class'] = class_name
# temp[main_column] = subclass_name

# df = pd.merge(df,temp,left_on='inchi_key',right_index=True,how='left')

# df

In [24]:
df.loc[df['node_id']=='41.0',all_columns]

,class_results,superclass_results,pathway_results
301,Monosaccharides,Saccharides,Carbohydrates


In [25]:

def min_additional_subgraph(graph, original_nodes,radius=1,min_count=0):
    nodes = []
    for node in original_nodes:
        # Create an ego graph centered at the current node
        
        ego_subgraph = nx.ego_graph(graph, node,radius=radius)
        nodes.extend(list(ego_subgraph.nodes))
    df = pd.DataFrame(data=nodes,columns=['nodes'])
    df = df.groupby('nodes').filter(lambda x: len(x) >= min_count)
    nodes = df['nodes'].tolist() + original_nodes
    
    nodes = list(set(nodes))
    nodes = sorted(nodes)
    return nodes

sc_df = pd.DataFrame()
sc_df['node_id'] = df['node_id'].values
sc_df[main_column] = df[main_column].values
for class_name in sc_df.loc[pd.notna(sc_df[main_column]),main_column].unique():
    idx = sc_df[main_column]==class_name
    original_nodes = sc_df.loc[idx,'node_id'].tolist()
    new_nodes = min_additional_subgraph(G, original_nodes,radius=2,min_count=3)
    sc_df[class_name] = False
    # idx = sc_df['node_id'].isin(original_nodes)
    # sc_df.loc[idx,class_name] = True
    idx = sc_df['node_id'].isin(new_nodes)
    sc_df.loc[idx,class_name] = True
sc_df.set_index('node_id',inplace=True)
sc_df.drop(columns=[main_column],inplace=True)
sc_df
# For each row in the DataFrame
# If the sum of the row is greater than 1
# Find the class with the highest frequency
# Set all other classes to False

,Branched fatty acids,Oxo fatty acids,Cinnamic acids and derivatives,Simple phenolic acids,Monosaccharides,Dicarboxylic acids,Acyl phloroglucinols,Menthane monoterpenoids,Rotenoids,Simple indole alkaloids,...,Phenylethanoids,Furofuranoid lignans,Biaryl type diarylheptanoids,Irregular monoterpenoids,Coumaronochromones,Estrane steroids,Pyrrolidine alkaloids,Valerenane sesquiterpenoids,Nitro fatty acids,Carboline alkaloids
node_id,,,,,,,,,,,,,,,,,,,,,
1033.0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0.0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3055.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4033.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5672.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10391.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10455.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8425.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
sc_df.loc['41.0']

Branched fatty acids              False
Oxo fatty acids                   False
Cinnamic acids and derivatives    False
Simple phenolic acids             False
Monosaccharides                    True
                                  ...  
Estrane steroids                  False
Pyrrolidine alkaloids             False
Valerenane sesquiterpenoids       False
Nitro fatty acids                 False
Carboline alkaloids               False
Name: 41.0, Length: 180, dtype: bool

In [27]:
# for columns with more than one True value, get the regional value_counts from an ego graph of radius 5 and replace with the most common value
for i,row in sc_df[sc_df.sum(axis=1)>1].iterrows():
    ego_subgraph = nx.ego_graph(G, i,radius=3) #was 5
    nodes = list(ego_subgraph.nodes)
    temp = sc_df.loc[nodes]
    temp = temp.sum()
    temp = temp[temp>0]
    if len(temp)>0:
        most_common_class = temp.idxmax()
        sc_df.loc[i] = False
        sc_df.loc[i,most_common_class] = True



In [28]:
# # make a list of the classes that are most common
# ranked_frequency = sc_df.sum(axis=0).sort_values(ascending=False)
# # for nodes that are in more than one class, only keep the most abundant class
# for i,row in sc_df[sc_df.sum(axis=1)>1].iterrows():
#     # get the most commonly occuring class
#     keep_class = ranked_frequency[row[row==True].index].sort_values(ascending=False).index[0]
#     # get the indices of all the assigned classes
#     idx = row[row==True].index
#     # drop all but the most common
#     idx = idx[idx!=keep_class]
#     # zero them out
#     sc_df.loc[i,idx] = False

In [29]:
sc_df.loc['246.0'].sort_values()
# sc_df.loc[246.0,'Amino acids, peptides, and analogues']

Branched fatty acids            False
Anthraquinones and anthrones    False
Isoflavones                     False
Plant xanthones                 False
Podocarpane diterpenoids        False
                                ...  
Quadrane sesquiterpenoids       False
Daucane sesquiterpenoids        False
Cembrane diterpenoids           False
Carboline alkaloids             False
Aminoacids                       True
Name: 246.0, Length: 180, dtype: bool

In [30]:
# for any class that has an ID in the original, reset it back to its original state
for i,row in df.iterrows():
    if pd.notna(row[main_column]):
        sc_df.loc[row['node_id'],:] = False
        sc_df.loc[row['node_id'],row[main_column]] = True


# recomute the ranked frequency now that ambiguous classes have been resolved
ranked_frequency = sc_df.sum(axis=0).sort_values(ascending=False)

In [31]:
cols = sc_df.columns
sc_df = sc_df.reset_index().melt(id_vars='node_id',value_vars=cols)
idx = sc_df['value']==True
sc_df = sc_df[idx]
sc_df = sc_df[pd.notna(sc_df['variable'])]
sc_df


,node_id,variable,value
0,1033.0,Branched fatty acids,True
84,8.0,Branched fatty acids,True
204,974.0,Branched fatty acids,True
208,1321.0,Branched fatty acids,True
241,620.0,Branched fatty acids,True
...,...,...,...
1306112,1328.0,Valerenane sesquiterpenoids,True
1306341,1524.0,Valerenane sesquiterpenoids,True
1313579,1411.0,Nitro fatty acids,True
1321009,1515.0,Carboline alkaloids,True


In [32]:
sc_df[sc_df['node_id']=='2533.0']

,node_id,variable,value


In [33]:
sc_df[sc_df['node_id']=='246.0']


,node_id,variable,value
89451,246.0,Aminoacids,True


In [34]:
sc_df.value_counts('variable').head(40)

variable
Aminoacids                                   1501
Prostaglandins                                791
Simple phenolic acids                         617
Dicarboxylic acids                            126
Simple coumarins                              115
Cinnamic acids and derivatives                 84
Chalcones                                      81
Flavones                                       71
Gibberellins                                   54
Other Octadecanoids                            53
Ursane and Taraxastane triterpenoids           49
Cholane steroids                               41
Hydroxy fatty acids                            33
Isoflavones                                    30
Anthraquinones and anthrones                   26
Arteminisin                                    25
Pyridine alkaloids                             22
Dipeptides                                     21
Oleanane triterpenoids                         21
Tetraketide meroterpenoids               

In [35]:
sc_df

,node_id,variable,value
0,1033.0,Branched fatty acids,True
84,8.0,Branched fatty acids,True
204,974.0,Branched fatty acids,True
208,1321.0,Branched fatty acids,True
241,620.0,Branched fatty acids,True
...,...,...,...
1306112,1328.0,Valerenane sesquiterpenoids,True
1306341,1524.0,Valerenane sesquiterpenoids,True
1313579,1411.0,Nitro fatty acids,True
1321009,1515.0,Carboline alkaloids,True


In [36]:
# Add cluster labels to graph
cluster_dict = sc_df.copy()
cluster_dict.rename(columns={'variable':'structural_cluster_subclassname'},inplace=True)
print(cluster_dict.shape)
if main_column != 'pathway_results':
    cluster_dict = pd.merge(df[all_columns].drop_duplicates(main_column),cluster_dict,left_on=main_column,right_on='structural_cluster_subclassname',how='right')
else:
    cluster_dict[main_column] = cluster_dict['structural_cluster_subclassname']

cluster_dict.drop(columns=['structural_cluster_subclassname','value'],inplace=True)
cluster_dict.drop_duplicates('node_id',inplace=True)
cluster_dict.set_index('node_id',inplace=True)
cluster_dict.fillna('',inplace=True)
cluster_dict = cluster_dict.add_suffix('_propagated')

cluster_dict

(4385, 3)


,class_results_propagated,superclass_results_propagated,pathway_results_propagated
node_id,,,
1033.0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids
8.0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids
974.0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids
1321.0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids
620.0,Branched fatty acids,Fatty Acids and Conjugates,Fatty acids
...,...,...,...
1328.0,Valerenane sesquiterpenoids,Sesquiterpenoids,Terpenoids
1524.0,Valerenane sesquiterpenoids,Sesquiterpenoids,Terpenoids
1411.0,Nitro fatty acids,Fatty Acids and Conjugates,Fatty acids


In [37]:
print(cluster_dict.shape)
# need to understand why there are duplicates

cluster_dict = cluster_dict.to_dict(orient='index')

nx.set_node_attributes(G, cluster_dict)
# cluster_dict
# nx.set_node_attributes(G, cluster_dict, 'mcs_structural_cluster_number')
# len(clusters),len(set(clusters))

(4385, 3)


In [38]:
df = G.nodes(data=True)
df = pd.DataFrame(df)
df.columns = ['node','data']
df['data'] = df['data'].apply(lambda x: dict(x))
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)
df.reset_index(inplace=True,drop=True)
df.value_counts('pathway_results')

pathway_results
Shikimates and Phenylpropanoids    499
Fatty acids                        304
Terpenoids                         242
Polyketides                        123
Amino acids and Peptides            95
Alkaloids                           58
Carbohydrates                       15
Name: count, dtype: int64

In [39]:
df.value_counts('pathway_results_propagated')

pathway_results_propagated
Amino acids and Peptides           1523
Shikimates and Phenylpropanoids    1129
Fatty acids                        1119
Terpenoids                          404
Polyketides                         135
Alkaloids                            60
Carbohydrates                        15
Name: count, dtype: int64

In [40]:
df.value_counts('pathway_results_propagated')

pathway_results_propagated
Amino acids and Peptides           1523
Shikimates and Phenylpropanoids    1129
Fatty acids                        1119
Terpenoids                          404
Polyketides                         135
Alkaloids                            60
Carbohydrates                        15
Name: count, dtype: int64

In [41]:
# df = G.nodes(data=True)
# df = pd.DataFrame(df)
# df.columns = ['node','data']
# df['data'] = df['data'].apply(lambda x: dict(x))
# df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)
# df.reset_index(inplace=True,drop=True)
# df['consensus_class'] = None

# # Count the number of occurrences of each subclass
# subclass_counts = df[main_column].value_counts()

# # Get the subclasses that have at least 100 values
# large_subclasses = subclass_counts[subclass_counts >= 100].index

# # Set the consensus_class for large subclasses
# df.loc[df[main_column].isin(large_subclasses), 'consensus_class'] = df.loc[df[main_column].isin(large_subclasses), main_column]

# # Get the subclasses that have less than 100 values
# small_subclasses = subclass_counts[subclass_counts < 100].index

# # Set the consensus_class for small subclasses
# df.loc[df[main_column].isin(small_subclasses), 'consensus_class'] = df.loc[df[main_column].isin(small_subclasses), 'class']

# # Get the subclasses that have less than 100 values
# class_counts = df['consensus_class'].value_counts()
# small_classes = class_counts[class_counts < 100].index

# # Set the consensus_class for small subclasses
# df.loc[df['consensus_class'].isin(small_classes), 'consensus_class'] = df.loc[df['consensus_class'].isin(small_classes), 'superclass']

# df = df[['node','consensus_class']]

# df.loc[df['consensus_class'].map(df['consensus_class'].value_counts()) < 100, 'consensus_class'] = None

# df.fillna('',inplace=True)
# df.value_counts('consensus_class')
# nx.set_node_attributes(G, df.set_index('node')['consensus_class'].to_dict(), 'consensus_class')


In [42]:
nx.write_graphml(G,'CarbonNetwork.graphml')
G_filt = nx.maximum_spanning_tree(G)
nx.write_graphml(G_filt, 'CarbonNetwork_mst.graphml')

In [43]:


import networkx as nx

# Get the connected components in the graph
components = list(nx.connected_components(G))

# Filter out subgraphs with less than 1000 nodes
filtered_components = [component for component in components if len(component) >= 1000]

# Create a new graph
G_filtered = nx.Graph()

# Add each subgraph to the new graph
for component in filtered_components:
    G_filtered = nx.compose(G_filtered, G.subgraph(component))
    
pos = nx.nx_agraph.graphviz_layout(G_filtered, prog='neato')

nx.draw(G_filtered, pos, with_labels=False, font_weight='bold', node_color='lightblue', node_size=3)
plt.title("Nodes associated with a structural cluster")
plt.show()

for node, (x, y) in pos.items():
    G_filtered.nodes[node]['x'] = x
    G_filtered.nodes[node]['y'] = y
nx.write_graphml(G_filtered, 'eom-net_0p1_subclass_clusters_mst_with_pos.graphml')


ValueError: Program neato not found in path.